In [7]:
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

from typesense import Client

from txtai.embeddings import Embeddings
from txtai.pipeline import Extractor

from rank_bm25 import BM25Okapi

2023-06-08 14:55:34.871508: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 14:55:35.103763: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-08 14:55:35.856714: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-08 14:55:35.856776: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

encoder = SentenceTransformer('all-mpnet-base-v2')

In [8]:
client = Client({
    'nodes': [{
        'host': 'localhost', # For Typesense Cloud use xxx.a1.typesense.net
        'port': '8108',      # For Typesense Cloud use 443
        'protocol': 'http'   # For Typesense Cloud use https
    }],
    'api_key': 'xyz',
    'connection_timeout_seconds': 5
})

In [9]:
embedding_column = 'mini_abstract_vector'
text_columns = ['abstract','keyphrase_summary','full_text']

In [10]:
query = 'murder'

In [11]:
client.operations.is_healthy()

True

In [12]:
client.collections.retrieve()

[{'created_at': 1685288257,
  'default_sorting_field': '',
  'enable_nested_fields': False,
  'fields': [{'facet': True,
    'index': True,
    'infix': False,
    'locale': '',
    'name': 'orgs',
    'optional': False,
    'sort': False,
    'type': 'string[]'},
   {'facet': True,
    'index': True,
    'infix': False,
    'locale': '',
    'name': 'person',
    'optional': False,
    'sort': False,
    'type': 'string[]'},
   {'facet': True,
    'index': True,
    'infix': False,
    'locale': '',
    'name': 'gpes',
    'optional': False,
    'sort': False,
    'type': 'string[]'},
   {'facet': True,
    'index': True,
    'infix': False,
    'locale': '',
    'name': 'keyphrases',
    'optional': False,
    'sort': False,
    'type': 'string[]'},
   {'facet': False,
    'index': True,
    'infix': False,
    'locale': '',
    'name': 'paragraphs',
    'optional': False,
    'sort': False,
    'type': 'string[]'},
   {'facet': False,
    'index': True,
    'infix': False,
    'loca

In [6]:
fts_query = {
    'q'         : query,
    'query_by'  : 'full_text,abstract,keyphrases,keyphrase_summary',
    'per_page'  : 250
}

fts_results = client.collections['statutes'].documents.search(fts_query)

fts_results

In [7]:
len(fts_results['hits'])

250

In [8]:
len(fts_results['hits'])

250

In [9]:
len(fts_results['hits'][1])

5

In [10]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [11]:
query_vec = ', '.join([str(x) for x in encoder.encode(query)])

In [12]:
search_requests = {
    'searches': [
        {
            'collection': 'statutes',
            'q' : '*',
            'vector_query': f'{embedding_column}:([{query_vec}], k:100)'
        }
    ]
}

# Search parameters that are common to all searches go here
common_search_params =  {}
vec_results = client.multi_search.perform(search_requests, common_search_params)['results'][0]
# vec_results = client.collections['statutes'].documents.search(fts_query)

vec_results

In [13]:
fts_results['hits'][0]['document']

{'abstract': "13-1103. Manslaughter; classification A. A person commits manslaughter by doing any of the following: 1. Recklessly causing the death of another person. 2. Committing second degree murder as prescribed in section 13-1104, subsection A on a sudden quarrel or heat of passion resulting from adequate provocation by the victim. 3. Intentionally providing the physical means that another person uses to die by suicide, with the knowledge that the person intends to die by suicide. 4. Committing second degree murder as prescribed in section 13-1104, subsection A, paragraph 3, while being coerced to do so by the use or threatened immediate use of unlawful deadly physical force on the person or a third person that a reasonable person in his situation would have been unable to resist. 5. Knowingly or recklessly causing the death of an unborn child by any physical injury to the mother. B. A person who is at least eighteen years of age commits manslaughter by intentionally providing adv

pd.DataFrame.from_dict(vec_results[0]['document'])

vec_results[0]['document']['url']

#### Vector index

#### BM25 FTS index

## Search

In [14]:
class QueryIndexTypesense():
    def __init__(self, vector_index, fts_index, client, embedding_column, encoder):
        self.vector_index = vector_index
        self.fts_index = fts_index
        # self.data = data
        self.embedding_column = embedding_column
        self.encoder = encoder
        self.client = client

    def format_fts_response(self, docs, columns=None):
        search_ids = dict()
        search_results = dict()

        print(f'number of fts docs: {len(docs)}')
        for position, doc in enumerate(docs, start=1):
            url = doc['document']['url']
            search_ids[url] = position
            search_results[url] = doc['document']

        return search_ids, search_results

    def format_vector_response(self, docs, columns=None):
        search_ids = dict()
        search_results = dict()
        print(f'number of vector docs: {len(docs)}')
        for position, doc in enumerate(docs, start=1):
            url = doc['document']['url']
            search_ids[url] = position
            search_results[url] = doc['document']

        return search_ids, search_results


    def search_bm25(self, query, fields='full_text,abstract,keyphrases,keyphrase_summary', top_k=10):
        fts_query = {
            'q'         : query,
            'query_by'  : 'full_text,keyphrase_summary,keyphrases',
            'per_page'  : top_k
        }

        self.bm25_results = self.client.collections[f'{self.fts_index}'].documents.search(fts_query)
        search_ids, search_results = self.format_fts_response(self.bm25_results['hits'])
        return search_ids, search_results

    def search_vector(self, query, top_k=10):
        query_vec = ', '.join([str(x) for x in self.encoder.encode(query)])

        search_requests = {
            'searches': [
                {
                    'collection': f'{self.vector_index}',
                    'q' : '*',
                    'vector_query': f'{self.embedding_column}:([{query_vec}], k:{top_k})',
                    'per_page'  : top_k
                }
            ]
        }

        # Search parameters that are common to all searches go here
        common_search_params =  {}
        self.vec_results = self.client.multi_search.perform(search_requests, common_search_params)['results'][0]['hits']
        search_ids, search_results = self.format_vector_response(self.vec_results)
        return search_ids, search_results

    def find_overlap(self, vector_ids, fts_ids):
        overlap = set(vector_ids.keys()).intersection(set(fts_ids.keys()))
        vector_overlap = {id:position for id, position in vector_ids.items() if id in overlap}
        fts_overlap = {id:position for id, position in fts_ids.items() if id in overlap}
        return vector_overlap, fts_overlap

    def rrf_reranking(self, vector_overlap, fts_overlap, fts_weight=1, vector_weight=1):
        overlap_ids = list(vector_overlap.keys())
        hybrid_rank = dict()
        for url in overlap_ids:
            vector_position = 1 / vector_overlap[url]
            fts_position = 1 / fts_overlap[url]
            hybrid_rank[url] = vector_position * vector_weight + fts_position * fts_weight

        hybrid_rank = [(k,v) for k,v in hybrid_rank.items()]
        hybrid_rank = sorted(hybrid_rank, key=lambda x: x[1])[::-1]
        return hybrid_rank

    def merge_parallel_indexes(self, vector_results, fts_results):
        vector_overlap, fts_overlap = self.find_overlap(vector_results[0], fts_results[0])
        self.reranking_idx = self.rrf_reranking(vector_overlap, fts_overlap)
        overlap_data = [vector_results[1][x[0]] for x in self.reranking_idx]
        return overlap_data


    def search(self, query, top_k_vector=100, top_k_fts=100):
        vector_results = self.search_vector(query, top_k=top_k_vector)
        fts_results = self.search_bm25(query, top_k=top_k_fts)

        print(len(vector_results))
        print(len(fts_results))
        
        self.overlap_results = self.merge_parallel_indexes(vector_results, fts_results)
        return self.overlap_results


In [15]:
class ResultQA():
    def __init__(self, context_documents, extractor_path, embedder_path):
        self.context = context_documents
        self.extractor_path = extractor_path
        self.embedder_path = embedder_path
        self.previous_question = ''
        self.current_answer = ''
        self.previous_query = ''

    def build_extractor(self):
        answer_embeddings = Embeddings({"path": self.embedder_path, "content": True})
        answer_embeddings.index((x, text, None) for x, text in enumerate(self.context))

        self.extractor = Extractor(answer_embeddings, self.extractor_path)

    def search_w_context(self, query):
        return self.search(f"{self.previous_question} {self.answer}. {query}")

    def prompt(self, question):
        return f"""Answer the following question using only the context below. Say 'no answer' when the question can't be answered.
    Question: {question}
    Context: """

    def search(self, query, question=None):
        if question is None:
            question = query
        self.previous_query = query
        self.previous_question = question
        self.answer = self.extractor([("answer", query, self.prompt(question), False)])[0][1]
        return self.answer

In [16]:
from sklearn import svm
from sentence_transformers import SentenceTransformer

st_embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def svm_rerank(query, texts):
    query_embedding = st_embedder.encode(query)
    text_embedding = st_embedder.encode(texts)
    x = np.concatenate([query_embedding[None, ...], text_embedding])
    y = np.zeros(x.shape[0])
    y[0] = 1

    clf = svm.LinearSVC(
        class_weight="balanced", verbose=False, max_iter=10000, tol=1e-6, C=0.1
    )
    clf.fit(x, y)

    similarities = clf.decision_function(x)
    sorted_ix = np.argsort(-similarities)
    
    zero_index = np.where(sorted_ix == 0)[0][0]
    if zero_index != 0:
        sorted_ix[0], sorted_ix[zero_index] = sorted_ix[zero_index], sorted_ix[0]

    denominator = np.max(similarities) - np.min(similarities) + 1e-6
    normalized_similarities = (similarities - np.min(similarities)) / denominator
            
    return similarities, sorted_ix

In [17]:
def weighted_rrf(ranks_df, sub_col, obj_col, output_col='wrrf', a=1, b=1, c=1, a_e=1, b_e=1, c_e=1):
    ranks_df[output_col] = ranks_df.apply(lambda x: _calculate_rrf(x[sub_col], x[obj_col],
                                                               a=a, b=b, c=c, a_e=a_e, b_e=b_e, c_e=c_e),
                                      axis=1)
    return ranks_df

def _calculate_rrf(subordinate_rank, objective_rank, a=1, b=1, c=1, a_e=1, b_e=1, c_e=1):
    return -1 * (a*(1 / subordinate_rank)**a_e + b*(1 / objective_rank)**b_e + c*( (1 / subordinate_rank) * (1 / objective_rank))**c_e)

In [18]:
queryer_codes = QueryIndexTypesense(vector_index='statutes',
                                    fts_index='statutes',
                                    encoder=encoder,
                                    embedding_column=embedding_column,
                                    client=client)

queryer_legislation = QueryIndexTypesense(vector_index='legislation',
                                          fts_index='legislation',
                                          encoder=encoder,
                                          embedding_column=embedding_column,
                                          client=client)

queryer_statutes = QueryIndexTypesense(vector_index='statutes',
                                          fts_index='statutes',
                                          encoder=encoder,
                                          embedding_column=embedding_column,
                                          client=client)

In [19]:
query = 'retirement police'

In [20]:
codes_results = queryer_codes.search(query, top_k_fts=250, top_k_vector=250)

number of vector docs: 250
number of fts docs: 250
2
2


In [21]:
legislation_results = queryer_legislation.search(query, top_k_fts=250, top_k_vector=250)

number of vector docs: 250
number of fts docs: 250
2
2


In [22]:
statute_results = queryer_statutes.search(query, top_k_fts=250, top_k_vector=250)

number of vector docs: 250
number of fts docs: 250
2
2


In [23]:
statute_results = pd.DataFrame.from_dict(statute_results)

In [24]:
statute_results

,abstract,full_text,id,keyphrase_summary,keyphrases,mini_abstract_vector,toc_depth_1,toc_depth_2,toc_depth_3,toc_depth_4,toc_depth_5,toc_depth_6,toc_depth_7,url
0,Section 45-8A-22.106 List of retired police an...,Universal Citation: AL Code § 45-8A-22.106 (20...,7730,[Section 45-8A-22.106 List of retired police a...,"[retired police, police department, fire depar...","[-0.04443212, 0.05896934, -0.025755543, -0.015...",Title 45 - Local Laws.,Chapter 8A - Calhoun County Municipalities.,Article 2 - Anniston.,Part 3 - Employees.,Division 3 - Police and Firefighters Retiremen...,Section 45-8A-22.106 - List of Retired Police ...,,https://law.justia.com/codes/alabama/2022/titl...
1,9-925. Computation of pension upon voluntary r...,Universal Citation: AZ Rev Stat § 9-925 (2022)...,79714,[In the absence of an application for retireme...,"[further police service, mandatory retirement,...","[-0.012091582, 0.07529851, -0.081745215, -0.00...",Title 9 - Cities and Towns,§ 9-925 - Computation of pension upon voluntar...,,,,,,https://law.justia.com/codes/arizona/2022/titl...
2,Section 45-37A-51.237 Payment of benefits duri...,Universal Citation: AL Code § 45-37A-51.237 (2...,3882,[(a) Anything herein to the contrary notwithst...,"[retired public safety officer, additional ret...","[-0.02587529, 0.06981341, 0.0033902, 0.0136635...",Title 45 - Local Laws.,Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,Part 2 - Retirement and Relief System.,Division 6 - Benefits.,Section 45-37A-51.237 - Payment of Benefits Du...,https://law.justia.com/codes/alabama/2022/titl...
3,Section 45-28A-42.71 List of retired policemen...,Universal Citation: AL Code § 45-28A-42.71 (20...,5291,[Such book shall also give a full and complete...,"[fire department, trustees, policemen, departm...","[-0.014028671, 0.033803258, -0.027991232, -0.0...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.71 - List of Retired Policem...,,https://law.justia.com/codes/alabama/2022/titl...
4,Section 36-21-8 Badge and pistol as part of re...,Universal Citation: AL Code § 36-21-8 (2022) S...,25096,"[Any person who, at the time of retirement, is...","[state capitol police officer, certain law enf...","[-0.01937215, 0.14755258, -0.05818062, 0.04057...",Title 36 - Public Officers and Employees.,Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8 - Badge and Pistol as Part of ...,,,,https://law.justia.com/codes/alabama/2022/titl...
5,Section 45-28A-42.61 Mandatory retirement. The...,Universal Citation: AL Code § 45-28A-42.61 (20...,5303,[Section 45-28A-42.61 Mandatory retirement. Th...,"[mandatory retirement, retirement, trustees, r...","[0.06873847, 0.07311056, -0.07359102, 0.007594...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.61 - Mandatory Retirement.,,https://law.justia.com/codes/alabama/2022/titl...
6,"Section 45-52A-32.01 Receipt of badge, pistol,...",Universal Citation: AL Code § 45-52A-32.01 (20...,2053,[The City of Decatur may provide each city pol...,"[city police officer, other retirement benefit...","[-0.04666455, 0.16111417, -0.032442562, -0.038...",Title 45 - Local Laws.,Chapter 52A - Morgan County Municipalities.,Article 3 - Decatur.,Part 3 - Employees.,"Section 45-52A-32.01 - Receipt of Badge, Pisto...",,,https://law.justia.com/codes/alabama/2022/titl...
7,Section 45-28A-42.62 Increase in retirement be...,Universal Citation: AL Code § 45-28A-42.62 (20...,5302,[Section 45-28A-42.62 Increase in retirement b...,"[retirement benefits, pension, pension fund, r...","[0.04446189, 0.04071646, -0.06711724, -0.00661...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Polic

In [25]:
len(queryer_codes.bm25_results['hits'])

250

In [26]:
code_bm25_urls = [x['document']['url'] for x in queryer_codes.bm25_results['hits']]

In [27]:
code_vec_urls = [x['document']['url'] for x in queryer_codes.vec_results]

In [28]:
legislation_bm25_urls = [x['document']['url'] for x in queryer_legislation.bm25_results['hits']]

In [29]:
legislation_vec_urls = [x['document']['url'] for x in queryer_legislation.vec_results]

In [80]:
code_df = pd.DataFrame.from_dict(codes_results)
code_df = code_df.drop_duplicates(subset=['url'])
code_df = code_df[~code_df['url'].str.contains('ocd')]
code_df = code_df.reset_index(drop=True)
code_df.shape

(45, 14)

In [81]:
legis_df = pd.DataFrame.from_dict(legislation_results)
legis_df = legis_df.drop_duplicates(subset=['url'])
legis_df.shape

(58, 14)

In [82]:
legis_df.head()

,abstract,full_text,id,keyphrase_summary,keyphrases,mini_abstract_vector,toc_depth_1,toc_depth_2,toc_depth_3,toc_depth_4,toc_depth_5,toc_depth_6,toc_depth_7,url
0,Stricken language would be deleted from and un...,Stricken language would be deleted from a...,23376,[Subtitle TO AMEND THE...,"[state police retirement system, retirement sy...","[-0.04053313, 0.089468345, -0.057196558, -0.00...",,,,,,,,ocd-bill/4b0a7de8-1879-44da-963f-9979e77172f1
1,Amendment No. 1 to House Bill 1343 Amend House...,Hall of the House of Representa...,23281,[In lieu of terminating employment and accepti...,[arkansas state police officers deferred optio...,"[-0.03640527, 0.09438813, -0.009914499, -0.025...",,,,,,,,ocd-bill/0a41f6cb-f120-4173-a732-02527674129f
2,Stricken language would be deleted from and un...,Stricken language would be deleted from ...,22598,"[Arkansas Code § 24-6-502(1)(B), concerning th...","[state police retirement system, arkansas stat...","[-0.0903515, 0.048025396, -0.034031156, 0.0075...",,,,,,,,ocd-bill/e76a939b-2a51-4497-a7ee-1e4dea552ba0
3,Be it enacted by the Legislature of the State ...,\n\nBe it enacted by the Legislature of the St...,16465,[Supplemental appropriations; public safety pe...,"[public safety personnel retirement system, co...","[0.051730845, 0.06026412, -0.08686618, 0.02246...",,,,,,,,ocd-bill/fb01a069-13a4-4e30-b16b-e53d022c1c2c
4,State of Arkansas 94th General Assembly A Bill...,State of Arkansas\n94th General Assembly ...,21532,[State of Arkansas 94th General Assembly ...,"[state police retirement system law, state pol...","[-0.06328863, 0.11919552, -0.0970095, -0.00531...",,,,,,,,ocd-bill/ee183e37-15fc-42de-ae9a-2fb3270b6706


In [83]:
from sklearn.metrics.pairwise import cosine_similarity

In [84]:
query_vec = encoder.encode(query)

In [85]:
code_embeddings = encoder.encode(code_df['abstract'].values)
code_similarities = cosine_similarity([query_vec], code_embeddings)
code_df['cosine_similarity'] = code_similarities[0]

In [86]:
legis_embeddings = encoder.encode(legis_df['abstract'].values)
legis_similarities = cosine_similarity([query_vec], legis_embeddings)
legis_df['cosine_similarity'] = legis_similarities[0]

In [87]:
code_df['result_rank'] = code_df.index.values + 1
legis_df['result_rank'] = legis_df.index.values + 1

In [88]:
code_df['cosine_rank'] = (-1*code_df['cosine_similarity']).rank()
legis_df['cosine_rank'] = (-1*legis_df['cosine_similarity']).rank()

In [89]:
code_df['rrf_rank'] = (1 / code_df['result_rank']) + (1 / code_df['cosine_rank'])
legis_df['rrf_rank'] = (1 / legis_df['result_rank']) + (1 / legis_df['cosine_rank'])

In [90]:
code_df = weighted_rrf(code_df, 'result_rank', 'cosine_rank', c_e=1)
legis_df = weighted_rrf(legis_df, 'result_rank', 'cosine_rank', a_e=1, b_e=1, c_e=1)

In [91]:
code_df

,abstract,full_text,id,keyphrase_summary,keyphrases,mini_abstract_vector,toc_depth_1,toc_depth_2,toc_depth_3,toc_depth_4,toc_depth_5,toc_depth_6,toc_depth_7,url,cosine_similarity,result_rank,cosine_rank,rrf_rank,wrrf
0,Section 45-8A-22.106 List of retired police an...,Universal Citation: AL Code § 45-8A-22.106 (20...,7730,[Section 45-8A-22.106 List of retired police a...,"[retired police, police department, fire depar...","[-0.04443212, 0.05896934, -0.025755543, -0.015...",Title 45 - Local Laws.,Chapter 8A - Calhoun County Municipalities.,Article 2 - Anniston.,Part 3 - Employees.,Division 3 - Police and Firefighters Retiremen...,Section 45-8A-22.106 - List of Retired Police ...,,https://law.justia.com/codes/alabama/2022/titl...,0.583167,1,1.0,2.000000,-3.000000
1,9-925. Computation of pension upon voluntary r...,Universal Citation: AZ Rev Stat § 9-925 (2022)...,79714,[In the absence of an application for retireme...,"[further police service, mandatory retirement,...","[-0.012091582, 0.07529851, -0.081745215, -0.00...",Title 9 - Cities and Towns,§ 9-925 - Computation of pension upon voluntar...,,,,,,https://law.justia.com/codes/arizona/2022/titl...,0.579776,2,2.0,1.000000,-1.250000
2,Section 45-37A-51.237 Payment of benefits duri...,Universal Citation: AL Code § 45-37A-51.237 (2...,3882,[(a) Anything herein to the contrary notwithst...,"[retired public safety officer, additional ret...","[-0.02587529, 0.06981341, 0.0033902, 0.0136635...",Title 45 - Local Laws.,Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,Part 2 - Retirement and Relief System.,Division 6 - Benefits.,Section 45-37A-51.237 - Payment of Benefits Du...,https://law.justia.com/codes/alabama/2022/titl...,0.567226,3,3.0,0.666667,-0.777778
3,Section 45-28A-42.71 List of retired policemen...,Universal Citation: AL Code § 45-28A-42.71 (20...,5291,[Such book shall also give a full and complete...,"[fire department, trustees, policemen, departm...","[-0.014028671, 0.033803258, -0.027991232, -0.0...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.71 - List of Retired Policem...,,https://law.justia.com/codes/alabama/2022/titl...,0.567134,4,4.0,0.500000,-0.562500
4,Section 36-21-8 Badge and pistol as part of re...,Universal Citation: AL Code § 36-21-8 (2022) S...,25096,"[Any person who, at the time of retirement, is...","[state capitol police officer, certain law enf...","[-0.01937215, 0.14755258, -0.05818062, 0.04057...",Title 36 - Public Officers and Employees.,Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8 - Badge and Pistol as Part of ...,,,,https://law.justia.com/codes/alabama/2022/titl...,0.564466,5,5.0,0.400000,-0.440000
5,Section 45-28A-42.61 Mandatory retirement. The...,Universal Citation: AL Code § 45-28A-42.61 (20...,5303,[Section 45-28A-42.61 Mandatory retirement. Th...,"[mandatory retirement, retirement, trustees, r...","[0.06873847, 0.07311056, -0.07359102, 0.007594...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.61 - Mandatory Retirement.,,https://law.justia.com/codes/alabama/2022/titl...,0.550516,6,6.0,0.333333,-0.361111
6,"Section 45-52A-32.01 Receipt of badge, pistol,...",Universal Citation: AL Code § 45-52A-32.01 (20...,2053,[The City of Decatur may provide each city pol...,"[city police officer, other retirement benefit...","[-0.04666455, 0.16111417, -0.032442562, -0.038...",Title 45 - Local Laws.,Chapter 52A - Morgan County Municipalities.,Article 3 - Decatur.,Part 3 - Employees.,"Section 45-52A-32.01 - Receipt of Badge, Pisto...",,,https://law.justia.com/codes/alabama/2022/titl...,0.548484,7,7.0,0.285714,-0.306122
7,Section 45-28A-42.62 Increase in retirement be...,Universal Citation: AL Code § 45-28A-42.6

In [92]:
result_df = pd.concat([code_df, legis_df], axis=0)

In [93]:
result_df.head().columns

Index(['abstract', 'full_text', 'id', 'keyphrase_summary', 'keyphrases',
       'mini_abstract_vector', 'toc_depth_1', 'toc_depth_2', 'toc_depth_3',
       'toc_depth_4', 'toc_depth_5', 'toc_depth_6', 'toc_depth_7', 'url',
       'cosine_similarity', 'result_rank', 'cosine_rank', 'rrf_rank', 'wrrf'],
      dtype='object')

In [94]:
result_df['wrrf_rank'] = result_df['wrrf'].rank()
result_df['merged_cosine_rank'] = (-1*result_df['cosine_similarity']).rank()

In [99]:
result_df.sort_values(by='wrrf_rank').tail(20)

,abstract,full_text,id,keyphrase_summary,keyphrases,mini_abstract_vector,toc_depth_1,toc_depth_2,toc_depth_3,toc_depth_4,...,toc_depth_6,toc_depth_7,url,cosine_similarity,result_rank,cosine_rank,rrf_rank,wrrf,wrrf_rank,merged_cosine_rank
38,"HR67 217134-1 By Representatives Treadaway, Sc...","HR67\n217134-1\nBy Representatives Treadaway, ...",6129,"[WHEREAS, highest commendations and congratula...","[west precinct patrol sergeant, birmingham pol...","[-0.06149006, 0.01284348, -0.007513092, 0.0135...",,,,,...,,,ocd-bill/41e274ec-14dc-462f-bb67-cdddb6cd63dc,0.315139,39,43.0,0.048897,-0.049493,84.5,88.0
38,Section 45-28A-42.53 Secretary-treasurer; cust...,Universal Citation: AL Code § 45-28A-42.53 (20...,5311,[Section 45-28A-42.53 Secretary-treasurer; cus...,"[treasurer, trustees, such treasurer, retireme...","[0.015568386, 0.038625598, -0.044880908, -0.02...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,...,Section 45-28A-42.53 - Secretary-Treasurer; Cu...,,https://law.justia.com/codes/alabama/2022/titl...,0.401476,39,43.0,0.048897,-0.049493,84.5,48.0
37,Section 45-37A-51.220 Normal retirement benefi...,Universal Citation: AL Code § 45-37A-51.220 (2...,3899,[Section 45-37A-51.220 Normal retirement benef...,"[retirement benefit, normal retirement benefit...","[-0.020739025, 0.06596159, 0.007822621, -0.054...",Title 45 - Local Laws.,Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,...,Division 6 - Benefits.,Section 45-37A-51.220 - Normal Retirement Bene...,https://law.justia.com/codes/alabama/2022/titl...,0.389104,38,45.0,0.048538,-0.049123,86.0,53.0
42,Section 45-28A-42.64 Application for disabilit...,Universal Citation: AL Code § 45-28A-42.64 (20...,5300,[Section 45-28A-42.64 Application for disabili...,"[disability retirement, pension, respective de...","[0.009175187, 0.058516875, -0.027929958, -0.01...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,...,Section 45-28A-42.64 - Application for Disabil...,,https://law.justia.com/codes/alabama/2022/titl...,0.409153,43,41.0,0.047646,-0.048213,87.0,45.0
49,HB396 181543-4. The bill would relate to any C...,HB396\n181543-4\nBy Representative Treadaway (...,4158,[HB396 181543-4 By Representative Treadaway (N...,"[jefferson county legislation first read, curr...","[0.007575608, 0.09169861, -0.030752044, -0.005...",,,,,...,,,ocd-bill/a0dfa6ef-64cf-44f2-88b6-4498b8655991,0.321278,50,37.0,0.047027,-0.047568,88.0,82.0
43,Section 45-37A-51.05 Retirement and relief fun...,Universal Citation: AL Code § 45-37A-51.05 (20...,3809,[This amount shall not exceed the amount neces...,"[pension system, city matching payments, retir...","[-0.05415747, 0.12365084, -0.013426754, 0.0080...",Title 45 - Local Laws.,Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,...,Section 45-37A-51.05 - Retirement and Relief F...,,https://law.justia.com/codes/alabama/2022/titl...,0.403115,44,42.0,0.046537,-0.047078,89.0,47.0
45,HB338 181530-1 by Representative Treadaway (N ...,HB338\n181530-1\nBy Representative Treadaway (...,4259,"[""(2) Member No. 1 shall be appointed by the J...","[jefferson county personnel board, retirees re...","[-0.004947779, 0.053378332, -0.06248222, 0.044...",,,,,...,,,ocd-bill/cb9046f2-d9a0-4e2b-8a4c-866a17f580e1,0.316048,46,42.0,0.045549,-0.046066,90.0,87.0
50,"Under existing law, there is no provision for ...",SB290\n205677-1\nBy Senator Ward\nRFD: Finance...,1873,[SB290 205677-1 By Senator Ward RFD: Finance a...,"[retirement allowances, retirement benefit, el...","[-0.0043558218, 0.0999703, -0.06805193, -0.013...",,,,,...,,,ocd-bill/293edf0b-750c-4596-976f-3db242887109,0.317133,51,39.0,0.045249,-0.045752,91.0,84.0
44,The survivor's benefit may be increased pursua...,Universal Citation: AL Code § 45-37A-51.228 (2...,3891,[A survivor to whom a benefit payment has been...,"[death benefit pay

In [46]:
result_df.sort_values(by=['wrrf_rank'])[['keyphrases','toc_depth_2','toc_depth_3','toc_depth_4','toc_depth_5','abstract','url']].head(10)

,keyphrases,toc_depth_2,toc_depth_3,toc_depth_4,toc_depth_5,abstract,url
0,"[retired police, police department, fire depar...",Chapter 8A - Calhoun County Municipalities.,Article 2 - Anniston.,Part 3 - Employees.,Division 3 - Police and Firefighters Retiremen...,Section 45-8A-22.106 List of retired police an...,https://law.justia.com/codes/alabama/2022/titl...
3,"[public safety personnel retirement system, co...",,,,,Be it enacted by the Legislature of the State ...,ocd-bill/fb01a069-13a4-4e30-b16b-e53d022c1c2c
0,"[state police retirement system, retirement sy...",,,,,Stricken language would be deleted from and un...,ocd-bill/4b0a7de8-1879-44da-963f-9979e77172f1
1,"[further police service, mandatory retirement,...",§ 9-925 - Computation of pension upon voluntar...,,,,9-925. Computation of pension upon voluntary r...,https://law.justia.com/codes/arizona/2022/titl...
2,"[retired public safety officer, additional ret...",Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,Part 2 - Retirement and Relief System.,Section 45-37A-51.237 Payment of benefits duri...,https://law.justia.com/codes/alabama/2022/titl...
7,"[alabama peace officers, state employees, tier...",,,,,HB240 180922-1 By Representative Treadaway RFD...,ocd-bill/c46c5bb7-fb5d-492b-bdb3-426cc28b509e
3,"[fire department, trustees, policemen, departm...",Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.71 List of retired policemen...,https://law.justia.com/codes/alabama/2022/titl...
1,[arkansas state police officers deferred optio...,,,,,Amendment No. 1 to House Bill 1343 Amend House...,ocd-bill/0a41f6cb-f120-4173-a732-02527674129f
9,[corrections corrections officer retirement pl...,,,,,Be it enacted by the Legislature of the State ...,ocd-bill/29bda3ca-c104-46ec-a7cf-d6d608d44538
5,"[state police retirement system, retirement sy...",,,,,State of Arkansas 94th General Assembly A Bill...,ocd-bill/d4312d23-d70e-4352-b4a3-f86868d7bd50


In [47]:
statute_results[['keyphrases','toc_depth_2','toc_depth_3','toc_depth_4','toc_depth_5','abstract','url']].head(10)

,keyphrases,toc_depth_2,toc_depth_3,toc_depth_4,toc_depth_5,abstract,url
0,"[retired police, police department, fire depar...",Chapter 8A - Calhoun County Municipalities.,Article 2 - Anniston.,Part 3 - Employees.,Division 3 - Police and Firefighters Retiremen...,Section 45-8A-22.106 List of retired police an...,https://law.justia.com/codes/alabama/2022/titl...
1,"[further police service, mandatory retirement,...",§ 9-925 - Computation of pension upon voluntar...,,,,9-925. Computation of pension upon voluntary r...,https://law.justia.com/codes/arizona/2022/titl...
2,"[retired public safety officer, additional ret...",Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,Part 2 - Retirement and Relief System.,Section 45-37A-51.237 Payment of benefits duri...,https://law.justia.com/codes/alabama/2022/titl...
3,"[fire department, trustees, policemen, departm...",Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.71 List of retired policemen...,https://law.justia.com/codes/alabama/2022/titl...
4,"[state capitol police officer, certain law enf...",Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8 - Badge and Pistol as Part of ...,,Section 36-21-8 Badge and pistol as part of re...,https://law.justia.com/codes/alabama/2022/titl...
5,"[mandatory retirement, retirement, trustees, r...",Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.61 Mandatory retirement. The...,https://law.justia.com/codes/alabama/2022/titl...
6,"[city police officer, other retirement benefit...",Chapter 52A - Morgan County Municipalities.,Article 3 - Decatur.,Part 3 - Employees.,"Section 45-52A-32.01 - Receipt of Badge, Pisto...","Section 45-52A-32.01 Receipt of badge, pistol,...",https://law.justia.com/codes/alabama/2022/titl...
7,"[retirement benefits, pension, pension fund, r...",Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.62 Increase in retirement be...,https://law.justia.com/codes/alabama/2022/titl...
8,"[state police retirement system, retirement sy...",,,,,State of Arkansas 94th General Assembly A Bill...,ocd-bill/d4312d23-d70e-4352-b4a3-f86868d7bd50
9,"[police department, officer, officers, service...",Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8.3 - Receipt of Badge or Pistol...,,Section 36-21-8.3 Receipt of badge or pistol u...,https://law.justia.com/codes/alabama/2022/titl...


In [48]:
(1/2)**-0.1

1.0717734625362931

In [49]:
result_df = weighted_rrf(result_df, 'wrrf_rank', 'merged_cosine_rank', 'merged_wrrf_score',
                         a=-1, b=-1, c=-1, a_e=-1, b_e=-1, c_e=-0.1)

In [50]:
result_df['merged_wrrf_rank'] = result_df['merged_wrrf_score'].rank()

In [51]:
result_df = result_df.sort_values(by=['merged_wrrf_score'], ascending=True)#.reset_index()

In [52]:
result_df.sort_values(by=['cosine_similarity'], ascending=False).head(10)

,abstract,full_text,id,keyphrase_summary,keyphrases,mini_abstract_vector,toc_depth_1,toc_depth_2,toc_depth_3,toc_depth_4,...,url,cosine_similarity,result_rank,cosine_rank,rrf_rank,wrrf,wrrf_rank,merged_cosine_rank,merged_wrrf_score,merged_wrrf_rank
0,Section 45-8A-22.106 List of retired police an...,Universal Citation: AL Code § 45-8A-22.106 (20...,7730,[Section 45-8A-22.106 List of retired police a...,"[retired police, police department, fire depar...","[-0.04443212, 0.05896934, -0.025755543, -0.015...",Title 45 - Local Laws.,Chapter 8A - Calhoun County Municipalities.,Article 2 - Anniston.,Part 3 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.583167,1,1.0,2.000000,-3.000000,1.0,1.0,3.000000,1.0
1,9-925. Computation of pension upon voluntary r...,Universal Citation: AZ Rev Stat § 9-925 (2022)...,79714,[In the absence of an application for retireme...,"[further police service, mandatory retirement,...","[-0.012091582, 0.07529851, -0.081745215, -0.00...",Title 9 - Cities and Towns,§ 9-925 - Computation of pension upon voluntar...,,,...,https://law.justia.com/codes/arizona/2022/titl...,0.579776,2,2.0,1.000000,-1.250000,4.0,2.0,7.231144,2.0
2,Section 45-37A-51.237 Payment of benefits duri...,Universal Citation: AL Code § 45-37A-51.237 (2...,3882,[(a) Anything herein to the contrary notwithst...,"[retired public safety officer, additional ret...","[-0.02587529, 0.06981341, 0.0033902, 0.0136635...",Title 45 - Local Laws.,Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.567226,3,3.0,0.666667,-0.777778,5.0,3.0,9.311019,3.0
3,Section 45-28A-42.71 List of retired policemen...,Universal Citation: AL Code § 45-28A-42.71 (20...,5291,[Such book shall also give a full and complete...,"[fire department, trustees, policemen, departm...","[-0.014028671, 0.033803258, -0.027991232, -0.0...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.567134,4,4.0,0.500000,-0.562500,7.0,4.0,12.395455,4.0
4,Section 36-21-8 Badge and pistol as part of re...,Universal Citation: AL Code § 36-21-8 (2022) S...,25096,"[Any person who, at the time of retirement, is...","[state capitol police officer, certain law enf...","[-0.01937215, 0.14755258, -0.05818062, 0.04057...",Title 36 - Public Officers and Employees.,Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8 - Badge and Pistol as Part of ...,...,https://law.justia.com/codes/alabama/2022/titl...,0.564466,5,5.0,0.400000,-0.440000,11.0,5.0,17.492919,5.0
5,Section 45-28A-42.61 Mandatory retirement. The...,Universal Citation: AL Code § 45-28A-42.61 (20...,5303,[Section 45-28A-42.61 Mandatory retirement. Th...,"[mandatory retirement, retirement, trustees, r...","[0.06873847, 0.07311056, -0.07359102, 0.007594...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.550516,6,6.0,0.333333,-0.361111,13.0,6.0,20.546000,6.0
6,"Section 45-52A-32.01 Receipt of badge, pistol,...",Universal Citation: AL Code § 45-52A-32.01 (20...,2053,[The City of Decatur may provide each city pol...,"[city police officer, other retirement benefit...","[-0.04666455, 0.16111417, -0.032442562, -0.038...",Title 45 - Local Laws.,Chapter 52A - Morgan County Municipalities.,Article 3 - Decatur.,Part 3 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.548484,7,7.0,0.285714,-0.306122,15.0,7.0,23.592645,7.0
7,Section 45-28A-42.62 Increase in retirement be...,Universal Citation: AL Code § 45-28A-42.62 (20...,5302,[Section 45-28A-42.62 Increase in retirement b...,"[retirement benefits, pension, pension fund, r...","[0.04446189, 0.04071646, -0.06711724, -0.00661...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,

In [53]:
result_df.sort_values(by=['wrrf'], ascending=True).head(10)

,abstract,full_text,id,keyphrase_summary,keyphrases,mini_abstract_vector,toc_depth_1,toc_depth_2,toc_depth_3,toc_depth_4,...,url,cosine_similarity,result_rank,cosine_rank,rrf_rank,wrrf,wrrf_rank,merged_cosine_rank,merged_wrrf_score,merged_wrrf_rank
0,Section 45-8A-22.106 List of retired police an...,Universal Citation: AL Code § 45-8A-22.106 (20...,7730,[Section 45-8A-22.106 List of retired police a...,"[retired police, police department, fire depar...","[-0.04443212, 0.05896934, -0.025755543, -0.015...",Title 45 - Local Laws.,Chapter 8A - Calhoun County Municipalities.,Article 2 - Anniston.,Part 3 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.583167,1,1.0,2.000000,-3.000000,1.0,1.0,3.000000,1.0
3,Be it enacted by the Legislature of the State ...,\n\nBe it enacted by the Legislature of the St...,16465,[Supplemental appropriations; public safety pe...,"[public safety personnel retirement system, co...","[0.051730845, 0.06026412, -0.08686618, 0.02246...",,,,,...,ocd-bill/fb01a069-13a4-4e30-b16b-e53d022c1c2c,0.438198,4,1.0,1.250000,-1.500000,2.0,29.0,32.500869,11.0
0,Stricken language would be deleted from and un...,Stricken language would be deleted from a...,23376,[Subtitle TO AMEND THE...,"[state police retirement system, retirement sy...","[-0.04053313, 0.089468345, -0.057196558, -0.00...",,,,,...,ocd-bill/4b0a7de8-1879-44da-963f-9979e77172f1,0.406307,1,5.0,1.200000,-1.400000,3.0,46.0,50.636771,19.0
1,9-925. Computation of pension upon voluntary r...,Universal Citation: AZ Rev Stat § 9-925 (2022)...,79714,[In the absence of an application for retireme...,"[further police service, mandatory retirement,...","[-0.012091582, 0.07529851, -0.081745215, -0.00...",Title 9 - Cities and Towns,§ 9-925 - Computation of pension upon voluntar...,,,...,https://law.justia.com/codes/arizona/2022/titl...,0.579776,2,2.0,1.000000,-1.250000,4.0,2.0,7.231144,2.0
2,Section 45-37A-51.237 Payment of benefits duri...,Universal Citation: AL Code § 45-37A-51.237 (2...,3882,[(a) Anything herein to the contrary notwithst...,"[retired public safety officer, additional ret...","[-0.02587529, 0.06981341, 0.0033902, 0.0136635...",Title 45 - Local Laws.,Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.567226,3,3.0,0.666667,-0.777778,5.0,3.0,9.311019,3.0
7,HB240 180922-1 By Representative Treadaway RFD...,HB240\n180922-1\nBy Representative Treadaway\n...,4255,[HB240 180922-1 By Representative Treadaway RF...,"[alabama peace officers, state employees, tier...","[-0.12643017, 0.03563939, -0.019043814, 0.0473...",,,,,...,ocd-bill/c46c5bb7-fb5d-492b-bdb3-426cc28b509e,0.419960,8,2.0,0.625000,-0.687500,6.0,36.0,43.711770,15.0
3,Section 45-28A-42.71 List of retired policemen...,Universal Citation: AL Code § 45-28A-42.71 (20...,5291,[Such book shall also give a full and complete...,"[fire department, trustees, policemen, departm...","[-0.014028671, 0.033803258, -0.027991232, -0.0...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.567134,4,4.0,0.500000,-0.562500,7.0,4.0,12.395455,4.0
1,Amendment No. 1 to House Bill 1343 Amend House...,Hall of the House of Representa...,23281,[In lieu of terminating employment and accepti...,[arkansas state police officers deferred optio...,"[-0.03640527, 0.09438813, -0.009914499, -0.025...",,,,,...,ocd-bill/0a41f6cb-f120-4173-a732-02527674129f,0.332250,2,31.0,0.532258,-0.548387,8.0,76.0,85.898412,39.0
9,Be it enacted by the Legislature of the State ...,\n\nBe it enacted by the Legislature of the St...,17450,[In addition to any other appropriations made ...,[corrections corrections officer retirement pl...,"[0.047022633, 0.06621971, -0.092565514, 0.0063...",,,,,...,ocd-bill/29bda3ca-c104-46ec-a7cf-d6d608d44538,0.418829,10,3.0,0.433333,-0.466667,9.0,37.0,47.787493,17.0
5,State of Arkansas 94th General Ass

In [73]:
result_df.sort_values(by=['merged_wrrf_score'], ascending=True).head(20)[['abstract','toc_depth_2','toc_depth_3', 'toc_depth_4', 'toc_depth_5', 'toc_depth_6',
                                                                          'toc_depth_7', 'url', 'cosine_similarity', 'result_rank', 'cosine_rank',
                                                                          'rrf_rank', 'wrrf', 'wrrf_rank', 'merged_cosine_rank',
                                                                          'merged_wrrf_score', 'merged_wrrf_rank']]

,abstract,toc_depth_2,toc_depth_3,toc_depth_4,toc_depth_5,toc_depth_6,toc_depth_7,url,cosine_similarity,result_rank,cosine_rank,rrf_rank,wrrf,wrrf_rank,merged_cosine_rank,merged_wrrf_score,merged_wrrf_rank
0,Section 45-8A-22.106 List of retired police an...,Chapter 8A - Calhoun County Municipalities.,Article 2 - Anniston.,Part 3 - Employees.,Division 3 - Police and Firefighters Retiremen...,Section 45-8A-22.106 - List of Retired Police ...,,https://law.justia.com/codes/alabama/2022/titl...,0.583167,1,1.0,2.000000,-3.000000,1.0,1.0,3.000000,1.0
1,9-925. Computation of pension upon voluntary r...,§ 9-925 - Computation of pension upon voluntar...,,,,,,https://law.justia.com/codes/arizona/2022/titl...,0.579776,2,2.0,1.000000,-1.250000,4.0,2.0,7.231144,2.0
2,Section 45-37A-51.237 Payment of benefits duri...,Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,Part 2 - Retirement and Relief System.,Division 6 - Benefits.,Section 45-37A-51.237 - Payment of Benefits Du...,https://law.justia.com/codes/alabama/2022/titl...,0.567226,3,3.0,0.666667,-0.777778,5.0,3.0,9.311019,3.0
3,Section 45-28A-42.71 List of retired policemen...,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.71 - List of Retired Policem...,,https://law.justia.com/codes/alabama/2022/titl...,0.567134,4,4.0,0.500000,-0.562500,7.0,4.0,12.395455,4.0
4,Section 36-21-8 Badge and pistol as part of re...,Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8 - Badge and Pistol as Part of ...,,,,https://law.justia.com/codes/alabama/2022/titl...,0.564466,5,5.0,0.400000,-0.440000,11.0,5.0,17.492919,5.0
5,Section 45-28A-42.61 Mandatory retirement. The...,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.61 - Mandatory Retirement.,,https://law.justia.com/codes/alabama/2022/titl...,0.550516,6,6.0,0.333333,-0.361111,13.0,6.0,20.546000,6.0
6,"Section 45-52A-32.01 Receipt of badge, pistol,...",Chapter 52A - Morgan County Municipalities.,Article 3 - Decatur.,Part 3 - Employees.,"Section 45-52A-32.01 - Receipt of Badge, Pisto...",,,https://law.justia.com/codes/alabama/2022/titl...,0.548484,7,7.0,0.285714,-0.306122,15.0,7.0,23.592645,7.0
7,Section 45-28A-42.62 Increase in retirement be...,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.62 - Increase in Retirement ...,,https://law.justia.com/codes/alabama/2022/titl...,0.539223,8,8.0,0.250000,-0.265625,16.0,8.0,25.624505,8.0
9,Section 45-28A-42.60 Forfeiture of benefits du...,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.60 - Forfeiture of Benefits ...,,https://law.justia.com/codes/alabama/2022/titl...,0.513236,10,9.0,0.211111,-0.222222,18.0,9.0,28.663227,9.0
8,Section 36-21-8.3 Receipt of badge or pistol u...,Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8.3 - Receipt of Badge or Pistol...,,,,https://law.justia.com/codes/alabama/2022/titl...,0.496948,9,11.0,0.202020,-0.212121,19.0,11.0,31.706140,10.0


In [78]:
result_df.sort_values(by=['merged_wrrf_score'], ascending=True).head(20)[['abstract','toc_depth_2','toc_depth_3', 'toc_depth_4', 'toc_depth_5', 'toc_depth_6',
                                                                                                                                  'toc_depth_7', 'url', 'cosine_similarity', 'result_rank', 'cosine_rank',
                                                                          'rrf_rank', 'wrrf', 'wrrf_rank', 'merged_cosine_rank',
                                                                          'merged_wrrf_score', 'merged_wrrf_rank']]

,abstract,toc_depth_2,toc_depth_3,toc_depth_4,toc_depth_5,toc_depth_6,toc_depth_7,url,cosine_similarity,result_rank,cosine_rank,rrf_rank,wrrf,wrrf_rank,merged_cosine_rank,merged_wrrf_score,merged_wrrf_rank
0,Section 45-8A-22.106 List of retired police an...,Chapter 8A - Calhoun County Municipalities.,Article 2 - Anniston.,Part 3 - Employees.,Division 3 - Police and Firefighters Retiremen...,Section 45-8A-22.106 - List of Retired Police ...,,https://law.justia.com/codes/alabama/2022/titl...,0.583167,1,1.0,2.000000,-3.000000,1.0,1.0,3.000000,1.0
1,9-925. Computation of pension upon voluntary r...,§ 9-925 - Computation of pension upon voluntar...,,,,,,https://law.justia.com/codes/arizona/2022/titl...,0.579776,2,2.0,1.000000,-1.250000,4.0,2.0,7.231144,2.0
2,Section 45-37A-51.237 Payment of benefits duri...,Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,Part 2 - Retirement and Relief System.,Division 6 - Benefits.,Section 45-37A-51.237 - Payment of Benefits Du...,https://law.justia.com/codes/alabama/2022/titl...,0.567226,3,3.0,0.666667,-0.777778,5.0,3.0,9.311019,3.0
3,Section 45-28A-42.71 List of retired policemen...,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.71 - List of Retired Policem...,,https://law.justia.com/codes/alabama/2022/titl...,0.567134,4,4.0,0.500000,-0.562500,7.0,4.0,12.395455,4.0
4,Section 36-21-8 Badge and pistol as part of re...,Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8 - Badge and Pistol as Part of ...,,,,https://law.justia.com/codes/alabama/2022/titl...,0.564466,5,5.0,0.400000,-0.440000,11.0,5.0,17.492919,5.0
5,Section 45-28A-42.61 Mandatory retirement. The...,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.61 - Mandatory Retirement.,,https://law.justia.com/codes/alabama/2022/titl...,0.550516,6,6.0,0.333333,-0.361111,13.0,6.0,20.546000,6.0
6,"Section 45-52A-32.01 Receipt of badge, pistol,...",Chapter 52A - Morgan County Municipalities.,Article 3 - Decatur.,Part 3 - Employees.,"Section 45-52A-32.01 - Receipt of Badge, Pisto...",,,https://law.justia.com/codes/alabama/2022/titl...,0.548484,7,7.0,0.285714,-0.306122,15.0,7.0,23.592645,7.0
7,Section 45-28A-42.62 Increase in retirement be...,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.62 - Increase in Retirement ...,,https://law.justia.com/codes/alabama/2022/titl...,0.539223,8,8.0,0.250000,-0.265625,16.0,8.0,25.624505,8.0
9,Section 45-28A-42.60 Forfeiture of benefits du...,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.60 - Forfeiture of Benefits ...,,https://law.justia.com/codes/alabama/2022/titl...,0.513236,10,9.0,0.211111,-0.222222,18.0,9.0,28.663227,9.0
8,Section 36-21-8.3 Receipt of badge or pistol u...,Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8.3 - Receipt of Badge or Pistol...,,,,https://law.justia.com/codes/alabama/2022/titl...,0.496948,9,11.0,0.202020,-0.212121,19.0,11.0,31.706140,10.0


In [77]:
statute_results.head(20)[['abstract','toc_depth_2','toc_depth_3', 'toc_depth_4', 'toc_depth_5', 'toc_depth_6',
'toc_depth_7', 'url']]

,abstract,toc_depth_2,toc_depth_3,toc_depth_4,toc_depth_5,toc_depth_6,toc_depth_7,url
0,Section 45-8A-22.106 List of retired police an...,Chapter 8A - Calhoun County Municipalities.,Article 2 - Anniston.,Part 3 - Employees.,Division 3 - Police and Firefighters Retiremen...,Section 45-8A-22.106 - List of Retired Police ...,,https://law.justia.com/codes/alabama/2022/titl...
1,9-925. Computation of pension upon voluntary r...,§ 9-925 - Computation of pension upon voluntar...,,,,,,https://law.justia.com/codes/arizona/2022/titl...
2,Section 45-37A-51.237 Payment of benefits duri...,Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,Part 2 - Retirement and Relief System.,Division 6 - Benefits.,Section 45-37A-51.237 - Payment of Benefits Du...,https://law.justia.com/codes/alabama/2022/titl...
3,Section 45-28A-42.71 List of retired policemen...,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.71 - List of Retired Policem...,,https://law.justia.com/codes/alabama/2022/titl...
4,Section 36-21-8 Badge and pistol as part of re...,Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8 - Badge and Pistol as Part of ...,,,,https://law.justia.com/codes/alabama/2022/titl...
5,Section 45-28A-42.61 Mandatory retirement. The...,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.61 - Mandatory Retirement.,,https://law.justia.com/codes/alabama/2022/titl...
6,"Section 45-52A-32.01 Receipt of badge, pistol,...",Chapter 52A - Morgan County Municipalities.,Article 3 - Decatur.,Part 3 - Employees.,"Section 45-52A-32.01 - Receipt of Badge, Pisto...",,,https://law.justia.com/codes/alabama/2022/titl...
7,Section 45-28A-42.62 Increase in retirement be...,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,Division 2 - Policemen and Firemen's Retiremen...,Section 45-28A-42.62 - Increase in Retirement ...,,https://law.justia.com/codes/alabama/2022/titl...
8,State of Arkansas 94th General Assembly A Bill...,,,,,,,ocd-bill/d4312d23-d70e-4352-b4a3-f86868d7bd50
9,Section 36-21-8.3 Receipt of badge or pistol u...,Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8.3 - Receipt of Badge or Pistol...,,,,https://law.justia.com/codes/alabama/2022/titl...


In [57]:
for _, doc in result_df[['url','abstract']].reset_index(drop=True).iterrows():
    print(f'\n-----{_}-{doc["url"]}------\n')
    print(doc['abstract'])


-----0-https://law.justia.com/codes/alabama/2022/title-45/chapter-8a/article-2/part-3/division-3/section-45-8a-22-106/------

Section 45-8A-22.106 List of retired police and firefighters. There shall be kept by the secretary-treasurer a book to be known as the List of Retired Police and Firefighters. The book shall give a full and complete history and record of the action of the retirement board in retiring any and all persons under the plan, including, but not limited to, each name, the date of entering service of the fire department or the police department, the date of retirement or other separation, and the reason for the retirement or separation. (Act 2012-484, p. 1349, §7.)

-----1-https://law.justia.com/codes/arizona/2022/title-9/section-9-925/------

9-925. Computation of pension upon voluntary retirement or upon mandatory retirement; limitation A. A member of the police department whose membership began prior to July 1, 1952 and who serves the department twenty years in the a

In [76]:
for _, doc in statute_results[['url','abstract']].reset_index(drop=True).head(20).iterrows():
    print(f'\n----{_}--{doc["url"]}------\n')
    print(doc['abstract'])


----0--https://law.justia.com/codes/alabama/2022/title-45/chapter-8a/article-2/part-3/division-3/section-45-8a-22-106/------

Section 45-8A-22.106 List of retired police and firefighters. There shall be kept by the secretary-treasurer a book to be known as the List of Retired Police and Firefighters. The book shall give a full and complete history and record of the action of the retirement board in retiring any and all persons under the plan, including, but not limited to, each name, the date of entering service of the fire department or the police department, the date of retirement or other separation, and the reason for the retirement or separation. (Act 2012-484, p. 1349, §7.)

----1--https://law.justia.com/codes/arizona/2022/title-9/section-9-925/------

9-925. Computation of pension upon voluntary retirement or upon mandatory retirement; limitation A. A member of the police department whose membership began prior to July 1, 1952 and who serves the department twenty years in the a

In [59]:
statutes_urls = statute_results['url'].unique()

In [60]:
len(statutes_urls)

49

In [61]:
overlap_df = result_df[result_df['url'].isin(statutes_urls)]

In [62]:
overlap_df

,abstract,full_text,id,keyphrase_summary,keyphrases,mini_abstract_vector,toc_depth_1,toc_depth_2,toc_depth_3,toc_depth_4,...,url,cosine_similarity,result_rank,cosine_rank,rrf_rank,wrrf,wrrf_rank,merged_cosine_rank,merged_wrrf_score,merged_wrrf_rank
0,Section 45-8A-22.106 List of retired police an...,Universal Citation: AL Code § 45-8A-22.106 (20...,7730,[Section 45-8A-22.106 List of retired police a...,"[retired police, police department, fire depar...","[-0.04443212, 0.05896934, -0.025755543, -0.015...",Title 45 - Local Laws.,Chapter 8A - Calhoun County Municipalities.,Article 2 - Anniston.,Part 3 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.583167,1,1.0,2.000000,-3.000000,1.0,1.0,3.000000,1.0
1,9-925. Computation of pension upon voluntary r...,Universal Citation: AZ Rev Stat § 9-925 (2022)...,79714,[In the absence of an application for retireme...,"[further police service, mandatory retirement,...","[-0.012091582, 0.07529851, -0.081745215, -0.00...",Title 9 - Cities and Towns,§ 9-925 - Computation of pension upon voluntar...,,,...,https://law.justia.com/codes/arizona/2022/titl...,0.579776,2,2.0,1.000000,-1.250000,4.0,2.0,7.231144,2.0
2,Section 45-37A-51.237 Payment of benefits duri...,Universal Citation: AL Code § 45-37A-51.237 (2...,3882,[(a) Anything herein to the contrary notwithst...,"[retired public safety officer, additional ret...","[-0.02587529, 0.06981341, 0.0033902, 0.0136635...",Title 45 - Local Laws.,Chapter 37A - Jefferson County Municipalities.,Article 5 - Birmingham.,Part 2 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.567226,3,3.0,0.666667,-0.777778,5.0,3.0,9.311019,3.0
3,Section 45-28A-42.71 List of retired policemen...,Universal Citation: AL Code § 45-28A-42.71 (20...,5291,[Such book shall also give a full and complete...,"[fire department, trustees, policemen, departm...","[-0.014028671, 0.033803258, -0.027991232, -0.0...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.567134,4,4.0,0.500000,-0.562500,7.0,4.0,12.395455,4.0
4,Section 36-21-8 Badge and pistol as part of re...,Universal Citation: AL Code § 36-21-8 (2022) S...,25096,"[Any person who, at the time of retirement, is...","[state capitol police officer, certain law enf...","[-0.01937215, 0.14755258, -0.05818062, 0.04057...",Title 36 - Public Officers and Employees.,Chapter 21 - Law Enforcement Officers Generally.,Article 1 - General Provisions.,Section 36-21-8 - Badge and Pistol as Part of ...,...,https://law.justia.com/codes/alabama/2022/titl...,0.564466,5,5.0,0.400000,-0.440000,11.0,5.0,17.492919,5.0
5,Section 45-28A-42.61 Mandatory retirement. The...,Universal Citation: AL Code § 45-28A-42.61 (20...,5303,[Section 45-28A-42.61 Mandatory retirement. Th...,"[mandatory retirement, retirement, trustees, r...","[0.06873847, 0.07311056, -0.07359102, 0.007594...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,Part 3 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.550516,6,6.0,0.333333,-0.361111,13.0,6.0,20.546000,6.0
6,"Section 45-52A-32.01 Receipt of badge, pistol,...",Universal Citation: AL Code § 45-52A-32.01 (20...,2053,[The City of Decatur may provide each city pol...,"[city police officer, other retirement benefit...","[-0.04666455, 0.16111417, -0.032442562, -0.038...",Title 45 - Local Laws.,Chapter 52A - Morgan County Municipalities.,Article 3 - Decatur.,Part 3 - Employees.,...,https://law.justia.com/codes/alabama/2022/titl...,0.548484,7,7.0,0.285714,-0.306122,15.0,7.0,23.592645,7.0
7,Section 45-28A-42.62 Increase in retirement be...,Universal Citation: AL Code § 45-28A-42.62 (20...,5302,[Section 45-28A-42.62 Increase in retirement b...,"[retirement benefits, pension, pension fund, r...","[0.04446189, 0.04071646, -0.06711724, -0.00661...",Title 45 - Local Laws.,Chapter 28A - Etowah County Municipalities.,Article 4 - Gadsden.,

In [63]:
qa_merged = ResultQA(context_documents=result_df['abstract'].values[:10],
              extractor_path= "google/flan-t5-small",
              embedder_path="sentence-transformers/all-MiniLM-L6-v2")

In [64]:
qa_merged.build_extractor()

In [65]:
qa_merged.search("What is the definition of an abortion?")

Token indices sequence length is longer than the specified maximum sequence length for this model (981 > 512). Running this sequence through the model will result in indexing errors


'abortion'

In [66]:
qa_merged.search_w_context('What are the reasonable justifications for an abortion?')

'No answer'

In [67]:
qa_merged.search('What are the reasonable justifications for an abortion?')

'no answer'

In [68]:
qa_statutes = ResultQA(context_documents=statute_results['abstract'].values[:10],
              extractor_path= "google/flan-t5-small",
              embedder_path="sentence-transformers/all-MiniLM-L6-v2")

In [69]:
qa_statutes.build_extractor()

In [70]:
qa_statutes.search("What is the definition of an abortion?")

Token indices sequence length is longer than the specified maximum sequence length for this model (746 > 512). Running this sequence through the model will result in indexing errors


'no answer'

In [71]:
qa_statutes.search_w_context('What are the reasonable justifications for an abortion?')

'no'

In [72]:
qa_statutes.search('What are the reasonable justifications for an abortion?')

'no answer'